# d6tstack with Dask

Dask is a great library for out-of-core computing. But if input files are not properly organized it quickly breaks. For example:

1) if columns are different between files, dask won't even read the data! It doesn't tell you what you need to do to fix it.

2) if column order is rearranged between files it will read data, but into the wrong columns and you won't notice it

Dask can't handle those scenarios. With d6tstack you can easily fix the situation with just a few lines of code!

For more instructions, examples and documentation see https://github.com/d6t/d6tstack

## Base Case: Columns are same between all files
As a base case, we have input files which have consistent input columns and thus can be easily read in dask.

In [22]:
import dask.dataframe as dd

# consistent format
ddf = dd.read_csv('test-data/input/test-data-input-csv-clean-*.csv')
ddf.compute()

,cost,date,profit,sales
0,-90,2011-02-01,110,200
1,-90,2011-02-02,110,200
2,-90,2011-02-03,110,200
3,-90,2011-02-04,110,200
4,-90,2011-02-05,110,200
5,-90,2011-02-06,110,200
6,-90,2011-02-07,110,200
7,-90,2011-02-08,110,200
8,-90,2011-02-09,110,200
9,-90,2011-02-10,110,200


## Problem Case 1: Columns are different between files
That worked well. But what happens if your input files have inconsistent columns across files? Say for example one file has a new column that the other files don't have.

In [23]:
# consistent format
ddf = dd.read_csv('test-data/input/test-data-input-csv-colmismatch-*.csv')
ddf.compute()


ValueError: Length mismatch: Expected axis has 5 elements, new values have 4 elements

## Fixing the problem with d6stack
Urgh! There's no way to use these files in dask. You don't even know what's going on. What file caused the problem? Why did it cause a problem? All you know is one file has more columns than the first file.

You can either manually process those files or use d6tstack to easily check for such a situation and fix it with a few lines of code - no manual processing required. Let's take a look!

In [24]:
import glob
import d6tstack.combine_csv

cfg_fnames = list(glob.glob('test-data/input/test-data-input-csv-colmismatch-*.csv'))
c = d6tstack.combine_csv.CombinerCSV(cfg_fnames, all_strings=True)

# check columns
print('all equal',c.is_all_equal())
print('')
c.is_col_present()

all equal False



,filename,cost,date,profit,profit2,sales
0,test-data-input-csv-colmismatch-feb.csv,True,True,True,False,True
1,test-data-input-csv-colmismatch-jan.csv,True,True,True,False,True
2,test-data-input-csv-colmismatch-mar.csv,True,True,True,True,True


Before using dask you can quickly use d6stack to check if all colums are consistent with `d6tstack.combine_csv.CombinerCSV.is_all_equal()`. If they are not consistent you can easily see which files are causing problems with `d6tstack.combine_csv.CombinerCSV.is_col_present()`, in this case there is a new column "profit2" in "test-data-input-csv-colmismatch-mar.csv".

**Let's use d6stack to fix the situation.** We will use out-of-core processing with `d6tstack.combine_csv.CombinerCSVAdvanced.combine_save()` to save data from all files into one combined file with constistent columns. Any missing data is filled with NaN (to keep only common columns use `cfg_col_sel=c.col_preview['columns_common']`) Just 2 lines of code!

In [25]:
# out-of-core combining
c2 = d6tstack.combine_csv.CombinerCSVAdvanced(c, cfg_col_sel=c.col_preview['columns_all'])
c2.combine_save('test-data/output/test-combined.csv')

True

Now you can read this in dask and do whatever you wanted to do in the first place.

In [26]:
# consistent format
ddf = dd.read_csv('test-data/output/test-combined.csv')
ddf.compute()

,profit,cost,date,profit2,sales,filename
0,110,-90,2011-02-01,NaN,200,test-data-input-csv-colmismatch-feb.csv
1,110,-90,2011-02-02,NaN,200,test-data-input-csv-colmismatch-feb.csv
2,110,-90,2011-02-03,NaN,200,test-data-input-csv-colmismatch-feb.csv
3,110,-90,2011-02-04,NaN,200,test-data-input-csv-colmismatch-feb.csv
4,110,-90,2011-02-05,NaN,200,test-data-input-csv-colmismatch-feb.csv
5,110,-90,2011-02-06,NaN,200,test-data-input-csv-colmismatch-feb.csv
6,110,-90,2011-02-07,NaN,200,test-data-input-csv-colmismatch-feb.csv
7,110,-90,2011-02-08,NaN,200,test-data-input-csv-colmismatch-feb.csv
8,110,-90,2011-02-09,NaN,200,test-data-input-csv-colmismatch-feb.csv
9,110,-90,2011-02-10,NaN,200,test-data-input-csv-colmismatch-feb.csv


## Problem Case 2: Columns are reordered between files
This is a sneaky case. The columns are the same but the order is different! Dask will read everything just fine without a warning but your data is totally messed up!

In the example below, the "profit" column contains data from the "cost" column!

In [27]:
# consistent format
ddf = dd.read_csv('test-data/input/test-data-input-csv-reorder-*.csv')
ddf.compute()

,date,sales,cost,profit
0,2011-02-01,200,-90,110
1,2011-02-02,200,-90,110
2,2011-02-03,200,-90,110
3,2011-02-04,200,-90,110
4,2011-02-05,200,-90,110
5,2011-02-06,200,-90,110
6,2011-02-07,200,-90,110
7,2011-02-08,200,-90,110
8,2011-02-09,200,-90,110
9,2011-02-10,200,-90,110


In [28]:
cfg_fnames = list(glob.glob('test-data/input/test-data-input-csv-reorder-*.csv'))
c = d6tstack.combine_csv.CombinerCSV(cfg_fnames, all_strings=True)

# check columns
col_preview = c.preview_columns()
print('all columns equal?' , col_preview['is_all_equal'])
print('')
print('in what order do columns appear in the files?')
print('')
col_preview['df_columns_order'].reset_index(drop=True)

all columns equal? False

in what order do columns appear in the files?



,filename,cost,date,profit,sales
0,test-data-input-csv-reorder-feb.csv,2,0,3,1
1,test-data-input-csv-reorder-jan.csv,2,0,3,1
2,test-data-input-csv-reorder-mar.csv,3,0,2,1


Again, just a useful check before loading data into dask you can see that the columns don't line up. It's very fast to run because it only reads the headers, there's NO reason for you NOT to do it from a QA perspective.

Same as above, the fix is the same few lines of code with d6stack.

In [29]:
# out-of-core combining
c2 = d6tstack.combine_csv.CombinerCSVAdvanced(c, cfg_col_sel=c.col_preview['columns_all'])
c2.combine_save('test-data/output/test-combined.csv')

True

In [30]:
# consistent format
ddf = dd.read_csv('test-data/output/test-combined.csv')
ddf.compute()

,date,sales,profit,cost,filename
0,2011-02-01,200,110,-90,test-data-input-csv-reorder-feb.csv
1,2011-02-02,200,110,-90,test-data-input-csv-reorder-feb.csv
2,2011-02-03,200,110,-90,test-data-input-csv-reorder-feb.csv
3,2011-02-04,200,110,-90,test-data-input-csv-reorder-feb.csv
4,2011-02-05,200,110,-90,test-data-input-csv-reorder-feb.csv
5,2011-02-06,200,110,-90,test-data-input-csv-reorder-feb.csv
6,2011-02-07,200,110,-90,test-data-input-csv-reorder-feb.csv
7,2011-02-08,200,110,-90,test-data-input-csv-reorder-feb.csv
8,2011-02-09,200,110,-90,test-data-input-csv-reorder-feb.csv
9,2011-02-10,200,110,-90,test-data-input-csv-reorder-feb.csv
